In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.dataquery import DataQueryClient as dqc
from tulip.data.dataquery import get_us_otr_yields
from tulip.plots import plot_lines, plot_line, switch_trace_to_secondary_axis

In [2]:
def analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker: str,
    domestic_yield_name: str,
    hedged_yield_series: pd.Series,
    hedged_yield_name: str,
    investor_region: str,
    yield_maturity: str,
    plot_source: str = "Kate Capital",
    redo: bool = False,
) -> None:
    """
    Analyzes and plots the difference between a hedged yield and a local domestic yield.

    Args:
        domestic_yield_ticker (str): Bloomberg ticker for the domestic yield.
        domestic_yield_name (str): Name to use for the domestic yield series (e.g., 'EUR 10Y Domestic Yield').
        hedged_yield_series (pd.Series): Pandas Series containing the hedged yield data.
                                         Expected to have a datetime index.
        hedged_yield_name (str): Name to use for the hedged yield series (e.g., 'US 10Y Hedged to Euros Yield').
        investor_region (str): The region of the investor (e.g., 'European', 'US').
        yield_maturity (str): The maturity of the yield (e.g., '10Y', '5Y').
        plot_source (str): Source to display on the plot. Defaults to 'Kate Capital'.

    Returns:
        None: Displays the plot directly.
    """
    # Fetch domestic yield data
    # This assumes 'bb' is a globally accessible Bloomberg data function or similar
    # In a real-world scenario, you might pass a Bloomberg session object or a function
    try:
        domestic_yield = (
            bb.bdh(domestic_yield_ticker, redo=redo)
            .iloc[:, 0]
            .rename(domestic_yield_name)
        )
    except Exception as e:
        print(f"Error fetching domestic yield data for {domestic_yield_ticker}: {e}")
        return

    # Ensure hedged_yield_series has a name
    if hedged_yield_series.name is None:
        hedged_yield_series = hedged_yield_series.rename(hedged_yield_name)
    else:
        # If a name is already set, we'll use it, or override if specified.
        # For this function, we'll override with hedged_yield_name for consistency.
        hedged_yield_series = hedged_yield_series.rename(hedged_yield_name)

    # Calculate the difference
    diff_series_name = "Hedged vs. Local"
    diff_for_yield = hedged_yield_series.sub(domestic_yield).rename(diff_series_name)

    # Concatenate into a DataFrame
    df = pd.concat(
        [domestic_yield, hedged_yield_series, diff_for_yield], axis=1
    ).dropna()

    # Create the plot title dynamically
    plot_title = f"<b> US</b> vs <b>Local {int(yield_maturity)}Y Yields </b>for a <b>{investor_region}</b> Investor"

    # Generate the initial plot
    fig = plot_lines(
        df, title=plot_title, tick_suffix="%", source=plot_source, show_0=True
    )

    # Switch the difference trace to a secondary axis
    switch_trace_to_secondary_axis(
        fig, diff_series_name, tick_suffix="%", as_area=True, show_0=True
    )

    # Display the plot
    fig.show()

In [3]:
us_yields = get_us_otr_yields()
forward_fx = dqc.get_fx(["EUR", "JPY", "GBP"], ["1Y"]).dropna(how="all")

eur_hedge_return = forward_fx.xs("EUR", axis=1).pow(-1)
eur_hedge_return = eur_hedge_return["1Y"].div(eur_hedge_return["Spot"]).add(-1)

jpy_hedge_return = forward_fx.xs("JPY", axis=1)
jpy_hedge_return = jpy_hedge_return["1Y"].div(jpy_hedge_return["Spot"]).add(-1)

gbp_hedge_return = forward_fx.xs("GBP", axis=1)
gbp_hedge_return = gbp_hedge_return["1Y"].div(gbp_hedge_return["Spot"]).add(-1)

relevant_tenors = ["12M", "2Y", "3Y", "5Y", "7Y", "10Y", "20Y", "30Y"]
jpn_investor_us_yields = us_yields.add(jpy_hedge_return.mul(100), axis=0)[
    relevant_tenors
].dropna(how="all")
eur_investor_us_yields = us_yields.add(eur_hedge_return.mul(100), axis=0)[
    relevant_tenors
].dropna(how="all")
gbp_investor_us_yields = us_yields.add(gbp_hedge_return.mul(100), axis=0)[
    relevant_tenors
].dropna(how="all")

2025-11-12 19:34:01.813346-0500 EST [INFO] pydataquery: DataQuery initialized with client_id='Dxwo8Fy5RZ50OkNd'


2025-11-12 19:34:01.850713-0500 EST [INFO] pydataquery: Set preferred_timezone='America/New_York'. current_time='2025-11-12 19:34:01.850713-0500 EST'


2025-11-12 19:34:05.391661-0500 EST [INFO] pydataquery: DataQuery initialized with client_id='Dxwo8Fy5RZ50OkNd'


2025-11-12 19:34:05.391661-0500 EST [INFO] pydataquery: Set preferred_timezone='America/New_York'. current_time='2025-11-12 19:34:05.391661-0500 EST'


### Hedged yields











We calculate the total USD return of a foreign‑currency bond when you hedge the FX risk assuming 1YR rolling hedge. We start with the bond’s local‑currency return and add the FX‑forward return—computed as the one‑year USD forward rate divided by today’s USD spot rate minus 1—which captures the currency gain or cost from the forward hedge. This captures the cross-currency bias that could exist in the FX cross.



$$
\mathrm{Hedged\ Return}
\;=\;
R_{\mathrm{bond}}^{\mathrm{foreign}}
\;+\;
\biggl(
\frac{F_{\mathrm{FX\,forward}}^{\mathrm{USD},\,1\mathrm{yr}}}
     {S_{\mathrm{FX}}^{\mathrm{USD},\,\mathrm{spot}}}
-1
\biggr)
$$



#### For a Japanese investor











In [4]:
pd.concat(
    [
        jpn_investor_us_yields.iloc[-1, :],
        jpn_investor_us_yields.iloc[-6, :].rename("Five Days Prior"),
        jpn_investor_us_yields.iloc[-20, :].rename("20 Days Prior"),
    ],
    axis=1,
).style.format("{:.2}%")

,2025-11-12 00:00:00,Five Days Prior,20 Days Prior
maturity,,,
12M,0.66%,0.68%,0.61%
2Y,0.54%,0.57%,0.46%
3Y,0.54%,0.59%,0.46%
5Y,0.65%,0.71%,0.58%
7Y,0.83%,0.89%,0.78%
10Y,1.1%,1.1%,1.0%
20Y,1.6%,1.7%,1.6%
30Y,1.6%,1.7%,1.6%


In [5]:
plot_lines(
    jpn_investor_us_yields,
    title="US Yields for a Japanese Hedged Investor",
    tick_suffix="%",
)

In [6]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTJPY1Y Govt",
    domestic_yield_name="JPN 1Y Domestic Yield",
    hedged_yield_series=jpn_investor_us_yields["12M"],
    hedged_yield_name="US 1Y Hedged to JPN Yield",
    investor_region="Japanese",
    yield_maturity=1,
    plot_source="Kate Capital",
)

In [7]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTJPY5Y Govt",
    domestic_yield_name="JPN 5Y Domestic Yield",
    hedged_yield_series=jpn_investor_us_yields["5Y"],
    hedged_yield_name="US 5Y Hedged to JPN Yield",
    investor_region="Japanese",
    yield_maturity=5,
    plot_source="Kate Capital",
)

In [8]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTJPY10Y Govt",
    domestic_yield_name="JPN 10Y Domestic Yield",
    hedged_yield_series=jpn_investor_us_yields["10Y"],
    hedged_yield_name="US 10Y Hedged to JPN Yield",
    investor_region="Japanese",
    yield_maturity=10,
    plot_source="Kate Capital",
)

#### For a European investor

In [9]:
pd.concat(
    [
        eur_investor_us_yields.iloc[-1, :],
        eur_investor_us_yields.iloc[-6, :].rename("Five Days Prior"),
        eur_investor_us_yields.iloc[-20, :].rename("20 Days Prior"),
    ],
    axis=1,
).style.format("{:.2}%")

,2025-11-12 00:00:00,Five Days Prior,20 Days Prior
maturity,,,
12M,2.0%,2.0%,1.9%
2Y,1.9%,1.9%,1.8%
3Y,1.9%,1.9%,1.8%
5Y,2.0%,2.0%,1.9%
7Y,2.2%,2.2%,2.1%
10Y,2.4%,2.4%,2.3%
20Y,3.0%,3.0%,2.9%
30Y,3.0%,3.0%,2.9%


In [10]:
plot_lines(
    eur_investor_us_yields,
    title="US Yields for a Euro Hedged Investor",
    tick_suffix="%",
)

In [11]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTDEM5Y Govt",
    domestic_yield_name="EUR 5Y Domestic Yield",
    hedged_yield_series=eur_investor_us_yields["5Y"],
    hedged_yield_name="US 5Y Hedged to Euros Yield",
    investor_region="Eurozone",
    yield_maturity=5,
    plot_source="Kate Capital",
)

In [12]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTDEM10Y Govt",
    domestic_yield_name="EUR 10Y Domestic Yield",
    hedged_yield_series=eur_investor_us_yields["10Y"],
    hedged_yield_name="US 10Y Hedged to Euros Yield",
    investor_region="Eurozone",
    yield_maturity=10,
    plot_source="Kate Capital",
)

#### For a UK investor

In [13]:
pd.concat(
    [
        gbp_investor_us_yields.iloc[-1, :],
        gbp_investor_us_yields.iloc[-6, :].rename("Five Days Prior"),
        gbp_investor_us_yields.iloc[-20, :].rename("20 Days Prior"),
    ],
    axis=1,
).style.format("{:.2}%")

,2025-11-12 00:00:00,Five Days Prior,20 Days Prior
maturity,,,
12M,3.6%,3.7%,3.3%
2Y,3.5%,3.6%,3.2%
3Y,3.5%,3.6%,3.2%
5Y,3.6%,3.7%,3.3%
7Y,3.8%,3.9%,3.5%
10Y,4.0%,4.1%,3.8%
20Y,4.6%,4.7%,4.3%
30Y,4.6%,4.7%,4.4%


In [14]:
plot_lines(
    eur_investor_us_yields,
    title="US Yields for a Euro Hedged Investor",
    tick_suffix="%",
)

In [15]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTGBP5Y Govt",
    domestic_yield_name="GBP 5Y Domestic Yield",
    hedged_yield_series=eur_investor_us_yields["5Y"],
    hedged_yield_name="GB 5Y Hedged to Euros Yield",
    investor_region="UK",
    yield_maturity=5,
    plot_source="Kate Capital",
)

In [16]:
analyze_and_plot_hedged_vs_local_yields(
    domestic_yield_ticker="GTGBP10Y Govt",
    domestic_yield_name="GBP 10Y Domestic Yield",
    hedged_yield_series=gbp_investor_us_yields["10Y"],
    hedged_yield_name="US 10Y Hedged to GBP Yield",
    investor_region="UK",
    yield_maturity=10,
    plot_source="Kate Capital",
)

In [17]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:34_